[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rycroft-group/math714/blob/main/k_spectral/spectral.ipynb)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, pi
import sys

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Optional: a library for plotting with LaTeX-like 
# styles nicer formatted figures
# Warning: need to have LaTeX installed
import scienceplots
plt.style.use(['science'])

# Custom helper function
from diff_matrices import diff_matrix
from cheb_matrix import cheb

# Spectral methods

## Accuracy of a spectral derivative

### Set up

In [ ]:
# Function to consider
def f(z):
    return 0.2*sin(z)+1/(2+cos(2*z))

# Derivative
def df(z):
    return 0.2*cos(z)+2*sin(2*z)/(2+cos(2*z))**2

### Assemble derivative matrix

In [ ]:
# Derivative mode
# mode = 1: order 2 centered-difference
# mode = 2: order 4 centered-difference
# mode = 3: spectral

# Assemble derivative matrix
mode = 3
n = 102
D = diff_matrix(n, mode)

# Look at the fixed matrix
plt.spy(D)
plt.show()

### Calculate derivatives and plot

In [ ]:
x = np.linspace(0, 2*pi, n, endpoint=False)
y = np.array([f(xx) for xx in x])

# Calculate derivative
# Order 2 centered-difference
D2 = diff_matrix(n, 1)
dy2 = np.dot(D2, y)
# Order 4 centered-difference
D4 = diff_matrix(n, 2)
dy4 = np.dot(D4, y)
# Spectral
Ds = diff_matrix(n, 3)
dys = np.dot(Ds, y)
# Analytical
dya = np.array([df(xx) for xx in x])

### Plot derivatives and errors

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(8, 6), dpi=300, sharex=True)

# Panel 1: Derivatives
axs[0].plot(x, dya, label='Analytical', color='k', linewidth=2)
axs[0].plot(x, dy2, label='Order 2', linestyle='--')
axs[0].plot(x, dy4, label='Order 4', linestyle='-.')
axs[0].plot(x, dys, label='Spectral', linestyle=':')
axs[0].set_ylabel('Derivative')
axs[0].legend()

# Panel 2: Errors
axs[1].plot(x, dy2 - dya, label='Order 2', linestyle='--')
axs[1].plot(x, dy4 - dya, label='Order 4', linestyle='-.')
axs[1].plot(x, dys - dya, label='Spectral', linestyle=':')
axs[1].set_xlabel('$x$')
axs[1].set_ylabel('Error')
axs[1].legend()

plt.tight_layout()
plt.show()

## Convergence of spectral methods

### Set up

In [ ]:
# Function to consider
def f(z):
    return 0.2*sin(z)+1/(2+cos(2*z))

# Derivative
def df(z):
    return 0.2*cos(z)+2*sin(2*z)/(2+cos(2*z))**2

# Assemble derivative matrix
def diff_error(n, mode):

    D = diff_matrix(n, mode)

    x = np.linspace(0, 2*pi, n, endpoint=False)
    y = np.array([f(xx) for xx in x])

    # Calculate derivative and plot
    dy = np.dot(D, y)
    err = np.array([dy[i]-df(x[i]) for i in range(n)])

    return np.linalg.norm(2*pi/n*err)

### Convergence study

In [ ]:
# Store results for each mode
results2 = []
results4 = []
resultss = []

n = 10
while n < 30000:
    # Order 2 centered-difference
    results2.append((n, 2*pi/n, diff_error(n, mode=1)))
    # Order 4 centered-difference
    results4.append((n, 2*pi/n, diff_error(n, mode=2)))
    # Spectral
    resultss.append((n, 2*pi/n, diff_error(n, mode=3)))
    n += 2*(n//4)

### Plot convergence rate

In [ ]:
# Extract grid size (N) and error for each method
N2, err2 = zip(*[(N, err) for N, _, err in results2])
N4, err4 = zip(*[(N, err) for N, _, err in results4])
Ns, errs = zip(*[(N, err) for N, _, err in resultss])

fig, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=300)
ax.loglog(N2, err2, label='Order 2', marker='o')
ax.loglog(N4, err4, label='Order 4', marker='s')
ax.loglog(Ns, errs, label='Spectral', marker='^')
ax.set_xlabel('$N$')
ax.set_ylabel('Error')
ax.legend()
ax.grid(True, which="both", ls=":")

plt.show()

###

## Transport equation

We demonstrate the Fourier spectral methods. Consider the transport equation
$$
  u_t + c(x) u_x = 0
$$
for $u(x,t)$ on the periodic interval $[0,2\pi)$ with the
spatially dependent speed
$$
  c(x) = \frac{1}{5} + \sin^2 (x-1)
$$
and the initial condition
$$
  u(x,0) = e^{-200(1-\cos(x-1))}.
$$

### Set up

In [ ]:
# Number of output snapshots
snaps = 40

# Number of iterations per snapshot
iters = 32

# Total grid points
n = 256
hn = n//2

# Grid spacing a time step size
h = 2*pi/n
dt = h/4

# Grid, initial condition, and previous step for use with the leapfrog method
x = np.linspace(0, 2*pi, n, endpoint=False)
u = np.exp(-200*(1-np.cos(x-1)))
u_prev = np.exp(-200*(1-np.cos(x-0.2*dt-1)))

# Speed
c = 0.2+np.square(np.sin(x-1))

### Fourier spectral derivatives

In [ ]:
# Spectral derivative
def du(u):
    z = np.fft.rfft(u)
    z[0:hn] *= 1j*np.arange(hn)
    z[hn] = 0
    return np.fft.irfft(z)

### Simulate

In [ ]:
# Store initial snapshot
z = np.empty((n, snaps+1))
z[:, 0] = u

for i in range(1, snaps+1):
    for j in range(iters):
        u_x = du(u)

        # Calculate next step
        uu = u_prev-2*dt*c*u_x

        # Update arrays
        u_prev[:] = u
        u[:] = uu
    z[:, i] = u

# Store snapshots
results = []
for j in range(n):
    e = [str(x[j])]
    for i in range(snaps+1):
        e.append(str(z[j, i]))
    results.append(" ".join(e))

### Visualize

In [ ]:
# Gradient color
colors = plt.cm.viridis(np.linspace(0, 1, snaps + 1))

fig, ax = plt.subplots(figsize=(8, 5), dpi=300)
ax.plot(x, z[:, 0], label='Initial', color=colors[0], lw=2)
for i in range(1, snaps):
    ax.plot(x, z[:, i], color=colors[i], lw=0.5)
ax.plot(x, z[:, snaps], label='Final', color=colors[snaps], lw=2)

ax.set_xlabel('$x$')
ax.set_ylabel('$u(x)$')
ax.set_title('Transport Equation Snapshots')
ax.legend()

plt.show()

In [ ]:
# Animation
fig, ax = plt.subplots(figsize=(8, 5), dpi=300)
line, = ax.plot(x, z[:, 0], color='b', lw=2)
ax.set_xlabel('$x$')
ax.set_ylabel('$u(x)$')
ax.set_title('Transport Equation Animation')
ax.set_ylim(np.min(z), np.max(z))

def update(frame):
    line.set_ydata(z[:, frame])
    ax.set_title(f'Transport Equation t={frame}')
    return line,

ani = FuncAnimation(fig, update, frames=snaps+1, interval=100, blit=True)
HTML(ani.to_jshtml())

## 1D boundary value problem

We demonstrate the Chebyshev spectral methods. Consider the linear ODE boundary value problem
$$
u_{xx} = e^{4x}, \qquad x\in[-1,1], \qquad u(\pm 1)=0.
$$
This is a Poisson problem with an anlytical solution
$$
u(x) = \left[ e^{4x} - x \sinh(4) - \cosh(4) \right] / 16.
$$

### Set up

In [ ]:
# Calculate spectral matrices
N = 64
x, D = cheb(N)
D2 = D@D

# Impose the Dirichlet boundary conditions
D2 = D2[1:-1, 1:-1]

# Define function
def f(x):
    return np.exp(4.*x[1:-1])

# Define exact solution
def u_exact(x):
    return (np.exp(4.*x) - x*np.sinh(4.) - np.cosh(4.)) / 16.

### Solve

In [ ]:
# Solve the system
u = np.linalg.solve(D2, f(x))
# Padding with boundary values
u = np.concatenate(([0], u, [0]))

### Plot

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=300)

ax.plot(x, u, label='Numerical', color='b', lw=2)
ax.scatter(x, u_exact(x), label='Analytical', color='r', marker='x')
ax.set_xlabel('$x$')
ax.set_ylabel('$u(x)$')
ax.legend()

plt.show()

## 2D boundary value problem

We continue demonstrating the Chebyshev spectral methods. Consider adding one more dimension to the previous ODE BVP
$$
u_{xx} + u_{yy} = 10\sin(8x(y-1)), \qquad -1<x,y<1, \qquad u=0\text{ on the boundary}.
$$

### Set up

In [ ]:
# Calculate spectral matrices
N = 12
x, D = cheb(N)
y = np.copy(x)

# Create the Kronecker product grid
# and impose the Dirichlet boundary conditions
xx, yy = np.meshgrid(x[1:-1], y[1:-1])
xx = xx.flatten()
yy = yy.flatten()
f = 10.*np.sin(8.*xx*(yy-1.))
D2 = D@D
D2 = D2[1:-1, 1:-1]
I = np.eye(N-1)

# Assemble the 2D Laplacian with Dirichlet BCs
L = np.kron(D2, I) + np.kron(I, D2)

In [ ]:
# Plot the Kronecker product grid
fig, ax = plt.subplots(figsize=(6, 6), dpi=300)
ax.scatter(xx, yy, s=10, color='k')
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
# Plot the 2D Laplacian
fig, ax = plt.subplots(figsize=(6, 6), dpi=300)
ax.spy(L)
ax.set_xticks([])
ax.set_yticks([])
plt.show()

### Solve

In [ ]:
# Solve the system
u = np.linalg.solve(L, f)
# Reshape skinny 1D vector back to 2D array
# and padding with boundary values
uu = np.zeros((N+1, N+1))
uu[1:-1, 1:-1] = u.reshape((N-1, N-1))
xx, yy = np.meshgrid(x, y)

### Plot

In [ ]:
fig = plt.figure(figsize=(8, 6), dpi=300)
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=30, azim=-135)

surf = ax.plot_surface(xx, yy, uu, cmap='viridis')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$u(x, y)$')
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10)
plt.show()